In [8]:
import ssl
import xmlrpc
import xmlrpc.client
import base64
import datetime

import requests
import urllib3
import json

import requests
import pandas as pd

import random
import string

In [9]:
### POST FUNCTIONS
def post_to_udl_json_data(udl_endpoint, username, unecrypted_password, json_data):
  #
  # perform udl interaction
  #
      key = username + ":" + unecrypted_password
      authkey = base64.b64encode(key.encode('utf-8')).decode("ascii")
      #creds = "Basic Y2hyaXN0aWFuLmtlbXBpczo5OTkwMCFDSUshY2lrISEh"
  
      udl_headers = {'accept': 'application/json',
                    'content-type': 'application/json',
                    #'Authorization': creds}
                    'Authorization': 'Basic {auth}'.format(auth=authkey)}
      print("Invoking {url} endpoint".format(url=udl_endpoint))
      print("calling with {data}".format(data=json_data))
  
      response = requests.post(udl_endpoint, 
                               data=json.dumps(json_data),
                               verify=False,
                               headers=udl_headers)
      response.raise_for_status()
      if response.ok:
          print("Completed data access at {url}".format(url=udl_endpoint))
      return response

def post_state_vector(udl_ep, udl_user, udl_pw, sync_pattern, time_stamp):
    try:
        state_vector_data = {
            "classificationMarking": "U",
            "msgType" : "LatencyTest",
            "source" : "MITRE",
            "msgBody" : {   "timeStamp" : time_stamp, 
                            "message" : "System latency test", 
                            "syncPattern" : sync_pattern
                        },
            "dataMode": "TEST"
        }

        print('Start executing UDL update for ' + udl_user)
        res = post_to_udl_json_data(udl_ep, udl_user, udl_pw, state_vector_data)
        print(res)
        print('Finished executing UDL update...')
    except xmlrpc.client.ProtocolError as err:
        print("A protocol error occurred")
        print("URL: %s" % err.url)
        print("HTTP/HTTPS headers: %s" % err.headers)
        print("Error code: %d" % err.errcode)
        print("Error message: %s" % err.errmsg)

In [10]:
### QUERY FUNCTIONS

# Returns a single data frame with the matching sync pattern or -1 if non were found
def ScanDataFrameForOnSyncPattern(data_frame, sync_pattern):
    dfDecrementer = data_frame.shape[0] - 1    # Get amount of rows in the dataframe
    stopSearch = False
    returnValue = -1

    while stopSearch == False:
        dataRow = data_frame.loc[dfDecrementer]

        if 'syncPattern' in dataRow['msgBody']:
            compString = dataRow['msgBody']['syncPattern']

            if(compString == sync_pattern):
                print(f"Test Message Found for: {sync_pattern}");
                returnValue = dataRow
                stopSearch = True;

        dfDecrementer -= 1

        if(dfDecrementer < 0):
            print(f"Did not find message for : {sync_pattern}")
            stopSearch = True
            break
    
    return returnValue

# Returns time difference between the local timep stmap and retrieved data frame
def GetServerCreationLatency(time_stamp, single_data_frame):
    serverTime = datetime.datetime.strptime(single_data_frame['createdAt'],"%Y-%m-%dT%H:%M:%S.%fZ")
    givenTime = datetime.datetime.strptime(time_stamp,"%Y-%m-%dT%H:%M:%S.%fZ")
    
    timeDifference = serverTime - givenTime

    print(f"Local Time: {givenTime}, Server Created Time: {serverTime}")
    print(f"Total Time Difference: {timeDifference}")

    return timeDifference

In [19]:
# Load All credentials
credsData = 0
with open('creds.json') as creds:
    credsData = json.load(creds)

user = credsData['user']
password = credsData['password']

service_endpoint_test = credsData['service_endpoint_test']
service_endpoint_production = credsData['service_endpoint_production']

url = credsData['url']
creds = credsData['creds']

In [12]:
# POST TO SERVER
syncPattern = ''.join(random.choices(string.ascii_letters + string.digits, k=16))
timeStamp = datetime.datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3]
timeStamp = timeStamp + 'Z'

post_state_vector(service_endpoint_test, user, password, syncPattern, timeStamp)

Start executing UDL update for christian.kempis
Invoking https://pep.bluestaq.com/udl/notification/ endpoint
calling with {'classificationMarking': 'U', 'msgType': 'LatencyTest', 'source': 'MITRE', 'msgBody': {'timeStamp': '2020-04-01T01:23:41.833Z', 'message': 'System latency test', 'syncPattern': 'pxNHRrZtw49VlUR3'}, 'dataMode': 'TEST'}
Completed data access at https://pep.bluestaq.com/udl/notification/
<Response [201]>
Finished executing UDL update...


In [20]:
### BEGIN QUERY FROM DATABASE
result = requests.get(url, headers={'Authorization':creds}, verify=False)
elsetsDataFrame = pd.DataFrame(result.json())

singleDataFrame = ScanDataFrameForOnSyncPattern(elsetsDataFrame, syncPattern)


Test Message Found for: pxNHRrZtw49VlUR3


In [21]:
singleDataFrame = ScanDataFrameForOnSyncPattern(elsetsDataFrame, syncPattern)
timeDifference = GetServerCreationLatency(timeStamp, singleDataFrame)

Test Message Found for: pxNHRrZtw49VlUR3
Local Time: 2020-04-01 01:23:41.833000, Server Created Time: 2020-04-01 01:23:42.363000
Total Time Difference: 0:00:00.530000
